In [189]:
import numpy as np
import pandas as pd
import nltk
import itertools
import random
#nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import pronouncing

In [2]:
ls data

clean_poems.csv      glove.840B.300d.txt  ptb.vocab.json
final_poems.csv      ptb.train.txt        ultimate_poems.csv
glove.6B.300d.txt    ptb.valid.txt


In [3]:
# prepare glove vectors
word_to_vec = {}
dim = 300
glove_filename = "data/glove.6B.300d.txt" # "./glove/glove.840B.300d.txt" vs 6B
with open(glove_filename) as file:
    for line in file:
        tokens = line.split()
        word, vector = tokens[0], np.array([float(v) for v in tokens[-300:]])
        if word not in word_to_vec:
            word_to_vec[word] = vector
print(len(word_to_vec))

400000


In [4]:
for syn in wn.synsets('rat'):
    print(syn.name(), end=" ")

rat.n.01 scab.n.01 rotter.n.01 informer.n.01 rat.n.05 rat.v.01 rat.v.02 fink.v.01 rat.v.04 rat.v.05 denounce.v.04 

In [135]:
def rhyming(word):
    return [word] + pronouncing.rhymes(word)

def cossim(v_main, v):
    return np.dot(v_main, v) / (np.linalg.norm(v_main) * np.linalg.norm(v))

def similar(word, word_to_vec, threshold, maxnum=-1):
    if word not in word_to_vec:
        return []
    similar = []
    v_main = word_to_vec[word]
    for k, v in word_to_vec.items():
        if cossim(v_main, v) > threshold:
            similar.append(k)
        if len(similar) == maxnum:
            return similar
    return similar

def synonyms(word, word_to_vec):
    syns = [] 
    for syn in wn.synsets(word):
        for l in syn.lemmas(): 
            if l.name() in word_to_vec:
                syns.append(l.name())
    return syns

def last_word(line, word_to_vec):
    tokens = line.split()
    for token in reversed(tokens):
        if token.isalnum() and token in word_to_vec: 
            return token
    return None

def find_match(words): # LOOKS FOR RHYME MATCHES BETWEEN WORDS DICT
    word_id = {}
    rhyme_id = {}
    for idx, word in enumerate(words):
        word_id[idx] = word
        rhyme_id[idx] = rhyming(word)
    matches = []
    for id1, id2 in itertools.combinations(range(len(words)), 2):
        if word_id[id1] in rhyme_id[id2]:
            matches.append([id1, id2])
    return matches

def replace_last(line, new_word, word_to_vec):
    tokens = line.split()
    new = ""
    replaced = False
    for token in reversed(tokens):
        if not replaced and token.isalnum() and token in word_to_vec:
            new = new_word + " " + new
            replaced = True
        else:
            new = token + " " + new
    return new

def generate_poem(lines, threshold=0.5, maxnum=-1, root=None): # SIMPLER VERSION
    line_id = {}
    final_word_id = {}
    rhymes_id = {}
    for idx, line in enumerate(lines):
        line_id[idx] = line
        final_word_id[idx] = last_word(line, word_to_vec)
        rhymes_id[idx] = rhyming(final_word_id[idx])
        
    pairs = []
    for id1, id2 in itertools.combinations(range(len(lines)), 2):
        pot1 = set(similar(final_word_id[id1], word_to_vec, threshold, maxnum))
        for word in pot1:
            if word in rhymes_id[id2]:
                pairs.append([replace_last(line_id[id1], word, word_to_vec), line_id[id2]])
        pot2 = set(similar(final_word_id[id2], word_to_vec, threshold, maxnum))
        for word in pot2:
            if word in rhymes_id[id1]:
                pairs.append([replace_last(line_id[id2], word, word_to_vec), line_id[id1]])
    return pairs

In [41]:
a = ["car", "far", "house", "star", "mouse"]
print(find_match(a))

[[0, 1], [0, 3], [1, 3], [2, 4]]


In [42]:
test_lines = ['the cat was in the house', 'it started chasing the rat', 'ate it and grew thick .']
result = generate_poem(test_lines)

In [43]:
print(result)

[['it started chasing the mouse ', 'the cat was in the house']]


In [67]:
yay1 = ["like an old nun",
"he finds the scent of the sphere",
"i ca n't remember a house of chicken soup",
"from the platform of a hill",
"of a song and fruits and mirth",
"rolled down into the ground",
"it has been nailed above , and has it",
"there ' s no <unk> - she has been given me",
"of the absolute singular",
"where you meet"]
yay2 = ["and voice and thinking of them",
"with bits of my papers : i ' ve been healed",
"then , as if you were",
"the world of snakes are dumb and another",
"the new year , and we ' d got a nerve",
"as if the soil were dried",
"till the wind ' s whiskey",
"she could not choose . but it lifts",
"before the calendar begins",
"where dogs wandered from the dead"]
yay3 = ["will answer the door and wide",
"she walks home . i ' m sure , and i took the very",
"in her eyes . i ' m not sure i ' m saying",
"and seek as bitter moments",
"when a young man wakes",
"of the same green rains",
"the world has gone after , the children say",
"when my son began to crawl",
"they are too many offices",
"and let the ape roam recognition"]
yay4 = ["for years on the chalk",
"at the same table ; i pictured her lovely to swallow",
"were some more wild and more sentence",
"or but it was a tool",
"who could make it tranquil , and yet the child",
"like being , but where we rode",
"every photograph at the same",
"sitting on a gardener ' s wearing grandpa ' s finger",
"of grass . i ' ve learned",
"the noise rose in the air"]
final_yay = ["and more than this western",
"all possibilities beneath our troubled",
"of dead muttering than that drifts",
"the fish i ' m in the office",
"to ask him to be ripped aside and cheer",
"to make it happiest , and that it was no more attempt",
"a body across the boundary",
"and they brought us down",
"something more than war",
"all is a jealous of genius",
"next door",
"the big tires of the cool brick",
"and nothing to make",
"on another . only a single scale",
"waiting for him to haunt the <unk>",
"the night and the birds were large food",
"and i ' m telling him he ' ll do",
"because of industry",
"with a chip of the dead",
"like the breaking of his falling body",
"what kind of idiot",
"( to go on the ground ) )",
"a little robin singing"
"and now they vanish",
"and the river flowed to the south",
"and let us soar from the starry faithful tiger",
"the morning is my favorite",
"such emptiness of this",
"a bird ' s cries flew out the farm",
"nothing but to do catching of my mind"]

In [47]:
print(yay)

['like an old nun', 'he finds the scent of the sphere', "i ca n't remember a house of chicken soup", 'from the platform of a hill', 'of a song and fruits and mirth', 'rolled down into the ground', 'it has been nailed above , and has it', "there ' s no <unk> - she has been given me", 'of the absolute singular', 'where you meet']


In [50]:
result = generate_poem(yay)

In [51]:
for pair in result:
    print(pair)

['it has been nailed above , and has done ', 'like an old nun']
['it has been nailed above , and has one ', 'like an old nun']
["there ' s no <unk> - she has been given fun ", 'like an old nun']
["there ' s no <unk> - she has been given done ", 'like an old nun']
["there ' s no <unk> - she has been given one ", 'like an old nun']
['it has been nailed above , and has clear ', 'he finds the scent of the sphere']
['it has been nailed above , and has year ', 'he finds the scent of the sphere']
['it has been nailed above , and has still ', 'from the platform of a hill']
['it has been nailed above , and has until ', 'from the platform of a hill']
['it has been nailed above , and has will ', 'from the platform of a hill']
['where you will ', 'from the platform of a hill']
['it has been nailed above , and has found ', 'rolled down into the ground']
['it has been nailed above , and has around ', 'rolled down into the ground']
['it has been nailed above , and has she ', "there ' s no <unk> - she

In [190]:
def soft_rhyme(line1, line2, threshold, synonym=True):
    word1, word2 = last_word(line1, word_to_vec), last_word(line2, word_to_vec)
    rhymes1, rhymes2 = rhyming(word1), rhyming(word2)
    if word1 in rhymes2: return [line1, line2]
    pairs = []
    if synonym:
        pot1, pot2 = set(synonyms(word1, word_to_vec)), set(synonyms(word2, word_to_vec))
        for w1, w2 in itertools.product(pot1, pot2):
            if w1 in rhyming(w2):  
                pairs.append([replace_last(line1, w1, word_to_vec), replace_last(line2, w2, word_to_vec)])
    else:
        pot1, pot2 = set(similar(word1, word_to_vec, threshold)), set(similar(word2, word_to_vec, threshold))

        for word in pot1:
            if word == word1: continue
            if word in rhymes2: pairs.append([replace_last(line1, word, word_to_vec), line2])
        for word in pot2:
            if word == word2: continue
            if word in rhymes1: pairs.append([line1, replace_last(line2, word, word_to_vec)])
                
    if not pairs:
        return [line1, line2]
    else:
        return random.choice(pairs)

In [173]:
def make_rhyme(poem): # FINAL FUNCTION
    temp_poem = []
    for i in range(1, len(poem), 2):
        temp_poem += soft_rhyme(poem[i-1], poem[i])
    new_poem = []
    for i in range(2, len(temp_poem), 3):
        new_lines = soft_rhyme(temp_poem[i-2], temp_poem[i])
        new_poem.append(new_lines[0])
        new_poem.append(temp_poem[i-1])
        new_poem.append(new_lines[1])
    return new_poem

In [191]:
def random_pairs(lines):
    order, pairs = [], []
    for i1, i2 in itertools.combinations(range(len(lines)), 2):
        order.append([i1, i2])
    shuffled = random.sample(order, len(order))
    for pair in shuffled:
        pairs.append(pair)
    return pairs

def slow_alter(lines, threshold=0.5):
    for pair in random_pairs(lines):
        i1, i2 = pair[0], pair[1]
        adjust = soft_rhyme(lines[i1], lines[i2], threshold, synonym=True)
        try: lines[i1], lines[i2] = adjust[0], adjust[1]
        except: print(adjust)
        for line in lines:
            print(line)
        print("----- -----")
    print("------ NEXT -----")
    for pair in random_pairs(lines):
        i1, i2 = pair[0], pair[1]
        adjust = soft_rhyme(lines[i1], lines[i2], threshold, synonym=False)
        try: lines[i1], lines[i2] = adjust[0], adjust[1]
        except: print(adjust)
        for line in lines:
            print(line)
        print("----- -----")

In [192]:
E89_1 = ["one of the other",
        "and the sound of my heart",
         "my heart beating my hand , my jaw held my bones",
         "no more human than human",
         "and my mother ' s face"
]
E89_2 = ["and the audience of the claw",
"they said what they do not know",
"i ' d rather be reluctant to be the ones",
"you can not be a clever person",
"and i am in the air"]
print(slow_alter(E89_2, threshold=0.5))

and the audience of the claw
they said what they do not know
i ' d rather be reluctant to be the ones
you can not be a clever person
and i am in the air
----- -----
and the audience of the claw
they said what they do not know
i ' d rather be reluctant to be the ones
you can not be a clever person
and i am in the air
----- -----
and the audience of the claw
they said what they do not know
i ' d rather be reluctant to be the ones
you can not be a clever person
and i am in the air
----- -----
and the audience of the claw
they said what they do not know
i ' d rather be reluctant to be the ones
you can not be a clever person
and i am in the air
----- -----
and the audience of the claw
they said what they do not know
i ' d rather be reluctant to be the ones
you can not be a clever person
and i am in the air
----- -----
and the audience of the claw
they said what they do not know
i ' d rather be reluctant to be the ones
you can not be a clever person
and i am in the air
----- -----
and the au

In [176]:
def synonyms(word, word_to_vec):
    syns = [] 
    for syn in wn.synsets(word):
        for l in syn.lemmas(): 
            if l.name() in word_to_vec:
                syns.append(l.name())
    return set(syns)

print(synonyms('good', word_to_vec))

{'respectable', 'good', 'honorable', 'skillful', 'dear', 'goodness', 'practiced', 'commodity', 'serious', 'safe', 'honest', 'near', 'just', 'well', 'full', 'effective', 'soundly', 'salutary', 'estimable', 'secure', 'adept', 'right', 'ripe', 'beneficial', 'expert', 'proficient', 'sound', 'upright', 'thoroughly', 'unspoilt', 'dependable', 'unspoiled', 'skilful'}


In [211]:
def pick_two(lines, word_to_vec, threshold=0.5):
    for pair in random_pairs(lines):
        i1, i2 = pair[0], pair[1] 
        result = soft_rhyme(lines[i1], lines[i2], threshold, synonym=True)
        w1, w2 = result[0], result[1]
        if w1 in rhyming(w2):
            return result
    return None

In [214]:
E99_4 = ["i have loved",
"and they ' re not to hold the breath of the passing",
"as if i ' d been happy",
"the dark line of the earth ' s chime appearing through the air",
"as if a small boy",
"the faint entrance , tires , and spin of the ferry",
"and lays",
"from some frail effort of worms",
"for the meek and the truth",
"i hear them bells in the echoing"]

E89 = E89_1 + E89_2

print(pick_two(final_yay, word_to_vec))

None


In [226]:
E99_1 = ["the wind is blowing",
     "personal question",
     "and i ' ll go down , i ' ll be vague , the current",
     "as if to see the glass",
     "and i ' d be seen , as if they were"]
E79_1 = ["she was to us . it is not the judgment",
     "as he stood in a bowl",
     "of an afterlife",
     "and then they said , the rolling blues",
     "on his own"]
E69_1 = ["and then they ' re traveling from dead",
     "then deep in thy blue and chilly room",
     "and nothing has occurred , but no scythe",
     "the front door , a pair of pearls . in a seat",
     "with sunset , and the sense of sense"]
E69_2 = ["it ' s starting to reach on the tip of your shoulders",
"i say , i will hear you in exchange",
"as i am exhausted",
"that loved me that never loved much",
"the pitcher received to urge"]
E59_1 = ["every trick has been thinking",
     "overhead , in clusters",
     "slowly . when i was born ",
     "the water . it washes ",
     "and my heart will feel if to feel"]
E59_2 = ["we do n't exist and we were not hungry , we",
"i ' ve seen to lie in earth",
"what they should ",
"from the city that hiding place",
"come and sliding down streets"]
E89_3 = ["and the very old man",
"it ' s not to be your own body",
"there ' s nothing more deadly than greatness",
"which is not",
"so a translucent prize"]
print(slow_alter(E89_3, threshold=0.5))

and the very old man
it ' s not to be your own body
there ' s nothing more deadly than greatness
which is not
so a translucent prize
----- -----
and the very old man
it ' s not to be your own body
there ' s nothing more deadly than greatness
which is not
so a translucent prize
----- -----
and the very old man
it ' s not to be your own body
there ' s nothing more deadly than greatness
which is not
so a translucent prize
----- -----
and the very old man
it ' s not to be your own body
there ' s nothing more deadly than greatness
which is not
so a translucent prize
----- -----
and the very old man
it ' s not to be your own body
there ' s nothing more deadly than greatness
which is not
so a translucent prize
----- -----
and the very old man
it ' s not to be your own body
there ' s nothing more deadly than greatness
which is not
so a translucent prize
----- -----
and the very old man
it ' s not to be your own body
there ' s nothing more deadly than greatness
which is not
so a translucent pri

In [223]:
def slow_syn_alter(lines, threshold=0.5, cap=3):
    for d in range(1,cap):
        for i in range(d, len(lines)):
            i1, i2 = i, i - d
            adjust = soft_rhyme(lines[i1], lines[i2], threshold, synonym=True)
            lines[i1], lines[i2] = adjust[0], adjust[1]
            for line in lines:
                print(line)
            print("----- -----")
print(slow_syn_alter(E69_2))

it ' s starting to reach on the tip of your shoulders
i say , i will hear you in switch 
as i am exhausted
that loved me that never loved which 
the pitcher received to itch 
----- -----
it ' s starting to reach on the tip of your shoulders
i say , i will hear you in switch 
as i am exhausted
that loved me that never loved which 
the pitcher received to itch 
----- -----
it ' s starting to reach on the tip of your shoulders
i say , i will hear you in switch 
as i am exhausted
that loved me that never loved which 
the pitcher received to itch 
----- -----
it ' s starting to reach on the tip of your shoulders
i say , i will hear you in switch 
as i am exhausted
that loved me that never loved which 
the pitcher received to itch 
----- -----
it ' s starting to reach on the tip of your shoulders
i say , i will hear you in switch 
as i am exhausted
that loved me that never loved which 
the pitcher received to itch 
----- -----
it ' s starting to reach on the tip of your shoulders
i say , i w